In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from pandas import read_csv
import multiprocessing
import os
import matplotlib.pyplot as plt
import seglearn
from sys import getsizeof
from scipy.stats import entropy, iqr, kurtosis
from scipy.stats.stats import pearsonr
from IPython.display import display
from ipywidgets import IntProgress
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score,r2_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from scipy.fftpack import fft
from scipy.signal import find_peaks
import random
from keras.layers import *
from keras.models import Model, Sequential
from keras.datasets import mnist
from keras.utils import to_categorical
from numpy.linalg import norm

In [ ]:
project_dir = "~/School/bbdc/"

## Load trained labels

In [ ]:
labels = read_csv(project_dir+"train/labels.train.csv", header=None)
labels.columns = ["Subject", "Start", "End", "Action"]
print(labels.shape)
labels.head()

In [ ]:
filenames = sorted(list(set([x.split(".")[0] for x in labels.Subject])))
filenames_test = ["s06t01","s06t02","s06t03","s06t04","s06t05"]

In [ ]:
label_mapping = {
    "la-object-pick": 0,
    "ra-object-pick": 0,
    "la-object-carry": 1,
    "ra-object-carry": 1,
    "la-object-place": 2,
    "ra-object-place": 2,
    "la-object-switch-hands": 3,
    "ra-object-switch-hands": 3,
    "la-object-orient": 4,
    "ra-object-orient": 4,
    "la-nothing": 5,
    "ra-nothing": 5
}
inverted_label_mapping_left = {
    0: "la-object-pick",
    1: "la-object-carry",
    2: "la-object-place",
    3: "la-object-switch-hands",
    4: "la-object-orient",
    5: "la-nothing"
}
inverted_label_mapping_right = {
    0: "ra-object-pick",
    1: "ra-object-carry",
    2: "ra-object-place",
    3: "ra-object-switch-hands",
    4: "ra-object-orient",
    5: "ra-nothing"
}
emg_positions = ["fa-o-t-r","fa-i-t-r","fa-i-b-r","fa-o-b-r","fa-o-t-l","fa-i-t-l","fa-i-b-l","fa-o-b-l"]

In [ ]:
def load_all_emg_data(filenames,train=True):
    emg_datasets = []
    if train:
        subdir = "train/"
    else:
        subdir = "test/"

    lengths = []
    f = IntProgress(min=0, max=len(filenames)-1)
    display(f)

    for i in range(len(filenames)):
        f.value += 1
        file_df = (pd.read_csv(project_dir+subdir+"emg/"+filenames[i]+".emg.csv"))
        emg_datasets.append(file_df)
        lengths.append(file_df.shape[0])

    emg_df = pd.concat(emg_datasets, ignore_index=True)
    print(emg_df.shape)
    return emg_df,lengths


In [ ]:
def load_all_mocap_data(filenames,train=True):
    mocap_datasets = []
    if train:
        subdir = "train/"
    else:
        subdir = "test/"

    lengths = []
    f = IntProgress(min=0, max=len(filenames)-1)
    display(f)

    for i in range(len(filenames)):
        f.value += 1
        file_df = (pd.read_csv(project_dir+subdir+"mocap/"+filenames[i]+".mocap.csv"))
        mocap_datasets.append(file_df)
        lengths.append(file_df.shape[0])

    mocap_df = pd.concat(mocap_datasets, ignore_index=True)
    print(mocap_df.shape)
    return mocap_df,lengths


In [ ]:
# Source: https://stackoverflow.com/questions/15722324/sliding-window-of-m-by-n-shape-numpy-ndarray
def window_stack(a, stepratio=1, width=5):
    # Step ratio is how much should consecutive windows overlap
    stepsize = int(stepratio*width)
    n = a.shape[0]
    return np.dstack( a[i:1+n+i-width:stepsize] for i in range(0,width) )

In [ ]:
def labeler(X,y,X_test=None,y_test=None):
    if X_test == None and y_test == None:
        print("No validation dataset passed, generating from given dataset")
        X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)
    else:
        X_train = X
        y_train = y        
    n_timesteps,n_features,n_outputs = X.shape[1], X.shape[2], y.shape[1]

    model = Sequential()
    model.add(Bidirectional(LSTM(512, input_shape=(n_timesteps,n_features))))
    # model.add(LSTM(512, input_shape=(n_timesteps,n_features)))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train,
                        epochs=10,
                        batch_size=128,
                        shuffle=False,
                        validation_data=(X_test, y_test))
    return model

In [ ]:
def flat_labeler(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)
    n_features,n_outputs = X.shape[1], y.shape[1]

    model = Sequential()
    model.add(Dense(512, input_shape=(n_features,)))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(X_train, y_train,
                        epochs=5,
                        batch_size=128,
                        validation_data=(X_test, y_test))
    return model

In [ ]:
def load_test_data():
    emg_df,lengths = load_all_emg_data(filenames=filenames_test,train=False)
    emg_features = emg_df.drop(["ts"],axis=1) # We don't need the time stamp for feature extraction
    # Normalize and offset negatives
    for columnName, column in emg_features.iteritems():
        if any(column<0):
            column += abs(min(column))
        emg_features[columnName] = column / column.max()
    
    emg_features_3d = window_stack(emg_features,stepratio=step_ratio,width=window_size)
    emg_features_3d = np.transpose(emg_features_3d, (0, 2, 1))
    print(emg_features_3d.shape)
    return emg_features_3d

    

In [ ]:
def load_test_data_mocap():
    mocap_df,lengths = load_all_mocap_data(filenames=filenames_test,train=False)
    mocap_features = mocap_df.drop(["ts"],axis=1) # We don't need the time stamp for feature extraction
    mocap_features = mocap_features.values
    nans, x= np.isnan(mocap_features), lambda z: z.nonzero()[0]
    mocap_features[nans]= np.interp(x(nans), x(~nans), mocap_features[~nans])
    mocap_features = pd.DataFrame(mocap_features)
    # Normalize and offset negatives
    for columnName, column in mocap_features.iteritems():
        if any(column<0):
            column += abs(min(column))
        mocap_features[columnName] = column / column.max()
    
    
    mocap_features_3d = window_stack(mocap_features,stepratio=step_ratio,width=window_size)
    mocap_features_3d = np.transpose(mocap_features_3d, (0, 2, 1))
    print(mocap_features_3d.shape,mocap_features.shape)
    return mocap_features_3d,lengths,mocap_features.values,mocap_df


In [ ]:
labels_test = read_csv(project_dir+"test/labels_test.csv",header=None)
labels_test.head

## Parameters and Constants

In [ ]:
# Constants
frequency = 600

# Parameters
window_size = frequency // 8 ## Samples per window
step_ratio = 0.2 ## Size of overlap from one window to another

# Train on EMG data

### Load labeled datasets

In [ ]:
emg_df,lengths = load_all_emg_data(filenames=filenames)

In [ ]:
emg_features = emg_df.drop(["ts","la","ra","la.1","ra.1"],axis=1) # We don't need the labels for feature extraction
print(emg_features.shape)

### Offset negative values and normalize

In [ ]:
for columnName, column in emg_features.iteritems():
    print(type(column),columnName)
    if any(column<0):
        column += abs(min(column))
    emg_features[columnName] = column / column.max()


### Perform Window Sampling

In [ ]:
emg_features_3d = window_stack(emg_features,stepratio=step_ratio,width=window_size)
# emg_features_3d = emg_features_3d.values
emg_features_3d = np.transpose(emg_features_3d, (0, 2, 1))
emg_features_3d.shape

In [ ]:
emg_features_flat = window_stack(emg_features,stepratio=step_ratio,width=window_size)
emg_features_flat.shape

### Prepare labels
Labels need to be in categorical manner, i.e. one hot encoded


In [ ]:
la_emg = np.array(emg_df['la'].map(label_mapping))
ra_emg = np.array(emg_df['ra'].map(label_mapping))

In [ ]:
start_index = 0
la_window_emg = []
ra_window_emg = []
# Iterate over each window and set the label based on the action that takes place the most
for i in range(1,emg_features_3d.shape[0]+2):
    end_index = int(min([window_size * step_ratio * i, emg_df.shape[0]]))

    count_l = np.bincount(la_emg[start_index:end_index])
    count_r = np.bincount(ra_emg[start_index:end_index])

    la_window_emg.append(np.argmax(count_l))
    ra_window_emg.append(np.argmax(count_r))
    start_index = end_index

la_window_emg = np.array(la_window_emg)
ra_window_emg = np.array(ra_window_emg)

In [ ]:
la_window_emg = la_window_emg[:emg_features_3d.shape[0]]
ra_window_emg = ra_window_emg[:emg_features_3d.shape[0]]
ra_window_emg.shape, la_window_emg.shape

In [ ]:
la_categorical_emg = to_categorical(la_window_emg)
ra_categorical_emg = to_categorical(ra_window_emg)


### Train labeling model

In [ ]:
right_model_emg = labeler(emg_features_3d,ra_categorical_emg)

In [ ]:
left_model_emg = labeler(emg_features_3d,la_categorical_emg)

### Predict training labels

In [ ]:
left_actions_pred_emg = left_model_emg.predict(emg_features_3d)
right_actions_pred_emg = right_model_emg.predict(emg_features_3d)

print(left_actions_pred_emg.shape,right_actions_pred_emg.shape)

In [ ]:
left_actions_pred_mapped_emg = np.argmax(left_actions_pred_emg,axis=1)
right_actions_pred_mapped_emg = np.argmax(right_actions_pred,axis=1)


### Get training accuracy

In [ ]:
display(
        accuracy_score(left_actions_pred_mapped_emg,la_window_emg),
        accuracy_score(right_actions_pred_mapped_emg,ra_window_emg)
)

### Load test data

In [ ]:
emg_features_3d_test = load_test_data()

### Predict on test data

In [ ]:
left_actions_pred_test_emg = left_model_emg.predict(emg_features_3d_test)
right_actions_pred_test_emg = right_model_emg.predict(emg_features_3d_test)


In [ ]:
left_actions_pred_mapped_test_emg = np.argmax(left_actions_pred_test_emg,axis=1)
right_actions_pred_mapped_test_emg = np.argmax(right_actions_pred_test_emg,axis=1)
left_actions_pred_mapped_test_emg,right_actions_pred_mapped_test_emg

### Parse predictions for output

In [ ]:
final_result = np.vstack((np.vectorize(inverted_label_mapping_left.get)(left_actions_pred_mapped_test),np.vectorize(inverted_label_mapping_right.get)(right_actions_pred_mapped_test)))
final_result = final_result.T

In [ ]:
final_result.shape

In [ ]:
df = pd.DataFrame(final_result,columns=["la","ra"])
df.to_csv(project_dir+"final_predictions.csv")

# Train on MoCap features

In [ ]:
mocap_df,lengths = load_all_mocap_data(filenames=filenames)

In [ ]:
mocap_features = mocap_df.drop(["ts","la","ra"],axis=1) # We don't need the labels for feature extraction
print(mocap_features.shape)

### Offset negative values and normalize

In [ ]:
for columnName, column in mocap_features.iteritems():
    if any(column<0):
        column += abs(min(column))
    mocap_features[columnName] = column / column.max()

mocap_features = mocap_features.values
nans, x= np.isnan(mocap_features), lambda z: z.nonzero()[0]
mocap_features[nans]= np.interp(x(nans), x(~nans), mocap_features[~nans])

### Perform Window Sampling

In [ ]:
mocap_features_3d_subjects = []
start_index = 0
end_index = 0
for i in lengths:
    end_index += i
    subject_3d = window_stack(mocap_features[start_index:end_index],stepratio=step_ratio,width=window_size)
    mocap_features_3d_subjects.append(subject_3d)
    start_index = end_index

In [ ]:
mocap_features_3d_subjects_train = np.transpose(np.vstack(mocap_features_3d_subjects[:-5]), (0,2,1))
mocap_features_3d_subjects_test = np.transpose(np.vstack(mocap_features_3d_subjects[-5:]), (0,2,1))

mocap_features_3d_subjects_train.shape,mocap_features_3d_subjects_test.shape

In [ ]:
train_size = mocap_features_3d_subjects_train.shape[0]
test_size = mocap_features_3d_subjects_test.shape[0]

In [ ]:
mocap_features_3d = window_stack(mocap_features,stepratio=step_ratio,width=window_size)
# mocap_features_3d = mocap_features_3d.values
mocap_features_3d = np.transpose(mocap_features_3d, (0, 2, 1))
mocap_features_3d.shape

### Prepare labels
Labels need to be in categorical manner, i.e. one hot encoded


In [ ]:
la_mocap = np.array(mocap_df['la'].map(label_mapping))
ra_mocap = np.array(mocap_df['ra'].map(label_mapping))

In [ ]:
start_index = 0
la_window_mocap = []
ra_window_mocap = []
for i in range(1,mocap_features_3d.shape[0]+2):
    end_index = int(min([int(window_size * step_ratio * i), mocap_df.shape[0]]))

    count_l = np.bincount(la_mocap[start_index:end_index])
    count_r = np.bincount(ra_mocap[start_index:end_index])

    if len(count_l):
        la_window_mocap.append(np.argmax(count_l))
    if len(count_r):
        ra_window_mocap.append(np.argmax(count_r))
    start_index = end_index

la_window_mocap = np.array(la_window_mocap)
ra_window_mocap = np.array(ra_window_mocap)

In [ ]:
la_window_mocap.shape, la_mocap.shape, mocap_features_3d.shape, window_size * step_ratio * 11842

In [ ]:
la_window_mocap = la_window_mocap[:mocap_features_3d.shape[0]]
ra_window_mocap = ra_window_mocap[:mocap_features_3d.shape[0]]
ra_window_mocap.shape, la_window_mocap.shape

In [ ]:
la_categorical_mocap = to_categorical(la_window_mocap)
ra_categorical_mocap = to_categorical(ra_window_mocap)


### Train labeling model

In [ ]:
la_categorical_mocap_train = la_categorical_mocap[:train_size]
ra_categorical_mocap_train = ra_categorical_mocap[:train_size]
la_categorical_mocap_test = la_categorical_mocap[train_size:]
ra_categorical_mocap_test = ra_categorical_mocap[train_size:]


In [ ]:
print(la_categorical_mocap_train.shape,
ra_categorical_mocap_train.shape,
la_categorical_mocap_test.shape,
ra_categorical_mocap_test.shape)

In [ ]:
right_model_subjects_mocap = labeler(mocap_features_3d_subjects_train,ra_categorical_mocap_train,mocap_features_3d_subjects_test,ra_categorical_mocap_test)

In [ ]:
left_model_subjects_mocap = labeler(mocap_features_3d_subjects_train,la_categorical_mocap_train,mocap_features_3d_subjects_test,la_categorical_mocap_test)

In [ ]:
right_model = labeler(mocap_features_3d,ra_categorical_mocap)

In [ ]:
left_model = labeler(mocap_features_3d,la_categorical_mocap)

### Predict on training features

In [ ]:
left_actions_pred = left_model.predict(mocap_features_3d)
right_actions_pred = right_model.predict(mocap_features_3d)


In [ ]:
left_actions_pred_mapped = np.argmax(left_actions_pred,axis=1)
right_actions_pred_mapped = np.argmax(right_actions_pred,axis=1)

In [ ]:
accuracy_score(left_actions_pred_mapped,la_window_mocap),accuracy_score(right_actions_pred_mapped,ra_window_mocap)

### Predict on test data

In [ ]:
mocap_features_3d_test,lengths_mocap_test,mocap_features_test,mocap_df_test    = load_test_data_mocap()

In [ ]:
left_actions_pred_test = left_model.predict(mocap_features_3d_test)
right_actions_pred_test = right_model.predict(mocap_features_3d_test)
print(left_actions_pred_test.shape, right_actions_pred_test.shape)

In [ ]:
left_actions_pred_mapped_test = np.argmax(left_actions_pred_test,axis=1)
right_actions_pred_mapped_test = np.argmax(right_actions_pred_test,axis=1)
left_actions_pred_mapped_test,right_actions_pred_mapped_test

### Parse predictions and output according to format

In [ ]:
final_result = np.vstack((np.vectorize(inverted_label_mapping_left.get)(left_actions_pred_mapped_test),np.vectorize(inverted_label_mapping_right.get)(right_actions_pred_mapped_test)))
final_result = final_result.T

In [ ]:
df = pd.DataFrame(final_result,columns=["la","ra"])
df.to_csv(project_dir+"final_predictions.csv")

In [ ]:
cur_length_index = 0
cur_length = lengths_mocap_test[cur_length_index]
times = []
file_names = []
row_count = 0
start_time = 0
for row in final_result:
    row_count += 1
    file_names.append("s06t0"+ str(cur_length_index+1))
    if row_count * 15 > cur_length:
        print("next file")
        row_count = 0
        cur_length_index += 1
        cur_length = lengths_mocap_test[cur_length_index]
        end_time = round(2 / 300 * lengths_mocap_test[cur_length_index-1], 3)
        times.append([start_time,end_time])
        start_time = 0
    else:
        end_time = round(2 / 300 * row_count * 15, 3)
        times.append([start_time,end_time])
        start_time = end_time





In [ ]:
final_final_result = np.concatenate([np.array(times),final_result],axis=1)
final_final_result.shape

In [ ]:
final_final_result = np.append(np.array(file_names).reshape(-1,1),final_final_result,axis=1)
final_final_result

In [ ]:
df = pd.DataFrame(final_final_result,columns=["subject1","start","stop","la","ra"])
df.to_csv(project_dir+"final_final_predictions4.csv")

## No window sampling
Predicts the action for measurements of a single time step

In [ ]:
mocap_features = mocap_features.values
mocap_features.shape

In [ ]:
la = np.array(mocap_df['la'].map(label_mapping))
ra = np.array(mocap_df['ra'].map(label_mapping))
la_categorical_flat = to_categorical(la)
ra_categorical_flat = to_categorical(ra)
la_categorical_flat.shape,ra_categorical_flat.shape

In [ ]:
left_model_flat = flat_labeler(mocap_features,la_categorical_flat)

In [ ]:
right_model_flat = flat_labeler(mocap_features,ra_categorical_flat)

In [ ]:
left_actions_pred_flat = left_model_flat.predict(mocap_features)
left_actions_pred_flat

In [ ]:
right_actions_pred_flat = right_model_flat.predict(mocap_features)
right_actions_pred_flat

In [ ]:
print(left_actions_pred_flat.shape)
left_actions_pred_flat_mapped = np.argmax(left_actions_pred_flat,axis=1)

In [ ]:
print(right_actions_pred_flat.shape)
right_actions_pred_flat_mapped = np.argmax(right_actions_pred_flat,axis=1)

In [ ]:
accuracy_score(left_actions_pred_flat_mapped,la),accuracy_score(right_actions_pred_flat_mapped,ra)

### Predict on test samples

In [ ]:
mocap_features_3d_test,lengths_mocap_test,mocap_features_test,mocap_df_test    = load_test_data_mocap()

In [ ]:
mocap_features_test.shape

In [ ]:
left_actions_pred_test_mocap = left_model_flat.predict(mocap_features_test)
left_actions_pred_test_mocap

In [ ]:
right_actions_pred_test_mocap = right_model_flat.predict(mocap_features_test)
right_actions_pred_test_mocap

In [ ]:
left_actions_pred_mapped_test_mocap_flat = np.argmax(left_actions_pred_test_mocap,axis=1)
right_actions_pred_mapped_test_mocap_flat = np.argmax(right_actions_pred_test_mocap,axis=1)
left_actions_pred_mapped_test_mocap_flat,right_actions_pred_mapped_test_mocap_flat

### Parse predictions into output format

In [ ]:
subjects = []
for i in range(len(lengths_mocap_test)):
    subjects += ["s06t0"+str(i+1)]*lengths_mocap_test[i]

In [ ]:
final_result = np.vstack((np.array(subjects),np.array(mocap_df_test['ts']),np.vectorize(inverted_label_mapping_left.get)(left_actions_pred_mapped_test_mocap_flat),np.vectorize(inverted_label_mapping_right.get)(right_actions_pred_mapped_test_mocap_flat)))
final_result = final_result.T

In [ ]:
final_result.shape

In [ ]:
final_result_right = []
final_result_left = []
templ = final_result[0]
tempr = final_result[0]
start_timel = 0
start_timer = 0
final_seconds = 0
for row in final_result:
    if row[2] != templ[2] or row[0] != templ[0]:
        end_timel = float(row[1]) if row[0] == templ[0] else final_seconds
        to_appendl = [templ[0]+".la", start_timel, end_timel, templ[2]]
        final_result_left.append(to_appendl)
        start_timel = end_timel
        templ = row
    if row[3] != tempr[3] or row[0] != tempr[0]:
        end_timer = float(row[1]) if row[0] == tempr[0] else final_seconds
        to_appendr = [tempr[0]+".ra", start_timer, end_timer, tempr[3]]
        final_result_right.append(to_appendr)
        start_timer = end_timer
        tempr = row
    final_seconds = float(row[1])

In [ ]:
final_result,final_result_right[-2:],len(final_result_right),len(final_result_left)

In [ ]:
final_result_truncated = np.vstack([np.array(final_result_left),np.array(final_result_right)])
final_result_truncated.shape

In [ ]:
df = pd.DataFrame(final_result_truncated,columns=["subject","start","end","action"])
df.to_csv(project_dir+"final_predictions_flat.csv")

# MoCap + EMG

In [ ]:
mocap_features.shape, emg_features.shape

### Method 1: Combine Feature Vectors

In [ ]:
quartized_emg_features = []
start_index = 0
for i in range(1,(emg_features.shape[0]//4)+1):
    end_index = i * 4
    if i == (emg_features.shape[0]//4):
        end_index = emg_features.shape[0]
    mean_value = emg_features[start_index:end_index].mean()
    quartized_emg_features.append(mean_value)
    start_index = end_index


In [ ]:
quartized_emg_features = np.array(quartized_emg_features)
quartized_emg_features.shape

In [ ]:
features_together = np.append(mocap_features[:443781],quartized_emg_features,axis=1)
features_together.shape

## Train labeling model

In [ ]:
features_together_3d = window_stack(features_together,stepratio=step_ratio,width=window_size)
# features_together_3d = features_together_3d.values
features_together_3d = np.transpose(features_together_3d, (0, 2, 1))
features_together_3d.shape

In [ ]:
la = np.array(mocap_df['la'].map(label_mapping))[:443781]
ra = np.array(mocap_df['ra'].map(label_mapping))[:443781]

In [ ]:
start_index = 0
la_window = []
ra_window = []
for i in range(1,features_together_3d.shape[0]+2):
    end_index = int(min([int(window_size * step_ratio * i), mocap_df.shape[0]]))

    count_l = np.bincount(la[start_index:end_index])
    count_r = np.bincount(ra[start_index:end_index])

    if len(count_l):
        la_window.append(np.argmax(count_l))
    if len(count_r):
        ra_window.append(np.argmax(count_r))
    start_index = end_index

la_window = np.array(la_window)
ra_window = np.array(ra_window)

In [ ]:
la_window.shape, la.shape, features_together_3d.shape, window_size * step_ratio * 11842

In [ ]:
la_window = la_window[:features_together_3d.shape[0]]
ra_window = ra_window[:features_together_3d.shape[0]]
ra_window.shape, la_window.shape

In [ ]:
la_categorical = to_categorical(la_window)
ra_categorical = to_categorical(ra_window)


In [ ]:
right_model_together = labeler(features_together_3d,ra_categorical)

In [ ]:
left_model_together = labeler(features_together_3d,la_categorical)

In [ ]:
left_actions_pred = left_model_together.predict(features_together_3d)
left_actions_pred

In [ ]:
right_actions_pred = right_model_together.predict(mocap_features_3d)
right_actions_pred

## Weight prediction based on accuracy

In [ ]:
mocap_right_accuracy = 53.03
mocap_left_accuracy = 55.68
emg_right_accuracy = 33.21
emg_left_accuracy = 43.2

In [ ]:
left_actions_pred.shape, left_actions_pred_emg.shape,right_actions_pred.shape, right_actions_pred_emg.shape


In [ ]:
left_actions_pred_mocap = left_actions_pred[:-1]
right_actions_pred_mocap = right_actions_pred[:-1]
left_actions_pred_test_emg

In [ ]:
mocap_selected = 0
emg_selected = 0
weighted_left_mocap = left_actions_pred_mocap * mocap_left_accuracy
weighted_right_mocap = right_actions_pred_mocap * mocap_right_accuracy
weighted_left_emg = left_actions_pred_emg * emg_left_accuracy
weighted_right_emg = right_actions_pred_emg * emg_right_accuracy


In [ ]:
weighted_left = np.append(weighted_left_mocap,weighted_left_emg,axis=1)
weighted_right = np.append(weighted_right_mocap,weighted_right_emg,axis=1)

In [ ]:
prediction_left = np.argmax(weighted_left,axis=1)%6
prediction_right = np.argmax(weighted_right,axis=1)%6

In [ ]:
la_window.shape, prediction_left.shape

In [ ]:
np.array_equal(la_window[:-1], la_window_emg)

In [ ]:
accuracy_score(prediction_left,la_window[:-1]),accuracy_score(prediction_right,ra_window[:-1])

In [ ]:
left_prediction_source = np.argmax(weighted_left,axis=1)//6
np.unique(left_prediction_source,return_counts=True)

In [ ]:
left_actions_pred_test.shape,left_actions_pred_test_emg.shape

In [ ]:
left_actions_pred_test_mocap = left_actions_pred_test
right_actions_pred_test_mocap = right_actions_pred_test


In [ ]:
mocap_selected = 0
emg_selected = 0
weighted_left_test_mocap = left_actions_pred_test_mocap * mocap_left_accuracy
weighted_right_test_mocap = right_actions_pred_test_mocap * mocap_right_accuracy
weighted_left_test_emg = left_actions_pred_test_emg * emg_left_accuracy
weighted_right_test_emg = right_actions_pred_test_emg * emg_right_accuracy


In [ ]:
print(weighted_left_test_mocap.shape)
print(weighted_right_test_mocap.shape)
print(weighted_left_test_emg.shape)
print(weighted_right_test_emg.shape)

In [ ]:
weighted_left_test = np.append(weighted_left_test_mocap,weighted_left_test_emg,axis=1)
weighted_right_test = np.append(weighted_right_test_mocap,weighted_right_test_emg,axis=1)

In [ ]:
prediction_test_left = np.argmax(weighted_left_test,axis=1)%6
prediction_test_right = np.argmax(weighted_right_test,axis=1)%6

In [ ]:
left_prediction_source_test = np.argmax(weighted_left_test,axis=1)//6
print(np.unique(left_prediction_source_test,return_counts=True))
right_prediction_source_test = np.argmax(weighted_right_test,axis=1)//6
print(np.unique(right_prediction_source_test,return_counts=True))